In [4]:
from dotenv import load_dotenv
import os 

load_dotenv(override=True)
api = os.getenv("API2")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")

from openai import OpenAI
client = OpenAI(api_key= api )

In [5]:
city = '서울'
response = client.chat.completions.create(
    model = default_model,
    messages=[{'role':'user',
            'content':f'오늘 {city} 날씨 알려줘'}],
    temperature=0
)

print(f'응답 데이터 : {response.choices[0].message.content}')

응답 데이터 : 죄송하지만, 저는 실시간 정보에 접근할 수 없습니다.  
오늘 서울의 최신 날씨를 확인하려면 네이버, 다음, 기상청, 또는 스마트폰의 날씨 앱을 이용해 주세요.

일반적으로 6월의 서울은 맑거나 흐리고, 낮 최고기온은 25~30도 사이, 습도가 높을 수 있습니다.  
정확한 정보는 실시간 날씨 서비스를 참고해 주세요!


In [9]:
#실제 코딩된 mock data만 반환하기

def get_weather(location):
    # 실제 API 연동 가능 (예: OpenWeatherMap)
    # 여기서는 예시로 mock data
    weather_data = {
        "Seoul": "맑고 기온은 24도입니다.",
        "New York": "비가 오고 있으며 기온은 15도입니다.",
        "Tokyo": "흐림, 기온은 20도입니다."
    }
    return weather_data.get(location, f"{location}의 날씨 정보를 찾을 수 없습니다.")

In [6]:
tools = [
    {
        "type":"function",
        "function" :{
            "name":"get_weather",
            "description":"오늘의 날씨를 알려주는 함수",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"날씨를 알고 싶은 지역"
                    },
                },
                "required":["location"]
            }
        }
    }
]

In [12]:
city = '서울'

messages = [{"role" : "user", 
            "content":f"{city} + 의 날씨는 어때?"}]
response = client.chat.completions.create(
    model=default_model,
    messages=messages,
    temperature=0,
    tools = tools, # 여기가 정의된 tools 의 목록을 보여주는 곳
    tool_choice = "auto"
)
print(f'응답 데이터 : {response.choices[0].message.content}')

응답 데이터 : None


In [10]:
# 사용 가능한 함수들을 딕셔너리로 매핑
available_functions = {
    "get_weather": get_weather
}

In [13]:
import json

if response.choices[0].message.tool_calls: # tool_calls가 있으면
    tool_call = response.choices[0].message.tool_calls[0]
    print(f'{tool_call.function.name}')
    print(f'{tool_call.function.arguments}')
    
    # 동적으로 함수 호출
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
        
    if function_name in available_functions:
        function_to_call = available_functions[function_name]
        weather_info = function_to_call(**function_args)
        print(weather_info)
    else:
        print(f"함수 {function_name}을 찾을 수 없습니다.")

get_weather
{"location":"서울"}
서울의 날씨 정보를 찾을 수 없습니다.


In [15]:
import json
from datetime import datetime

##########################################################################################
########################## step 1 tool로 정의할 function 정의하기 #########################
##########################################################################################

def get_weather(location):
    weather_data = {
        "서울": "맑고 기온은 24도입니다.",
        "뉴욕": "비가 오고 있으며 기온은 15도입니다.",
        "도쿄": "흐림, 기온은 20도입니다."
    }
    return weather_data.get(location, f"{location}의 날씨 정보를 찾을 수 없습니다.")

def get_time(timezone="Asia/Seoul"):
    # 간단한 시간 조회 함수
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"{timezone} 기준 현재 시간: {current_time}"

def calculate(operation, a, b):
    # 간단한 계산 함수
    if operation == "add":
        return f"{a} + {b} = {a + b}"
    elif operation == "subtract":
        return f"{a} - {b} = {a - b}"
    elif operation == "multiply":
        return f"{a} × {b} = {a * b}"
    elif operation == "divide":
        if b != 0:
            return f"{a} ÷ {b} = {a / b}"
        else:
            return "0으로 나눌 수 없습니다."
    else:
        return "지원하지 않는 연산입니다."

def search_info(query):
    # 간단한 정보 검색 함수 (mock)
    info_data = {
        "파이썬": "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다.",
        "AI": "인공지능(AI)은 기계가 인간의 지능을 모방하는 기술입니다.",
        "서울": "서울은 대한민국의 수도이며 약 950만 명이 거주합니다."
    }
    return info_data.get(query, f"'{query}'에 대한 정보를 찾을 수 없습니다.")

##########################################################################################
########################## step 2 tool 정의하기 ##########################################
##########################################################################################

# 사용 가능한 함수들을 딕셔너리로 매핑
available_functions = {
    "get_weather": get_weather,
    "get_time": get_time,
    "calculate": calculate,
    "search_info": search_info
}

# 여러 개의 도구 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "특정 지역의 오늘 날씨를 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "날씨를 알고 싶은 지역명"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "현재 시간을 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "시간대 (기본값: Asia/Seoul)",
                        "default": "Asia/Seoul"
                    }
                },
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "두 숫자의 사칙연산을 수행하는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {
                        "type": "string",
                        "description": "연산 종류 (add, subtract, multiply, divide)",
                        "enum": ["add", "subtract", "multiply", "divide"]
                    },
                    "a": {
                        "type": "number",
                        "description": "첫 번째 숫자"
                    },
                    "b": {
                        "type": "number",
                        "description": "두 번째 숫자"
                    }
                },
                "required": ["operation", "a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_info",
            "description": "특정 주제에 대한 기본 정보를 검색하는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색하고 싶은 주제나 키워드"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

##########################################################################################
########################## step 3 tool 불러오기 ##########################################
##########################################################################################


# 사용 예시
def process_user_request(user_message):
    messages = [{"role": "user", "content": user_message}]
    
    response = client.chat.completions.create(
        model=default_model,
        messages=messages,
        temperature=0,
        tools=tools,
        tool_choice="auto"
    )
    
    print(f'응답 데이터: {response.choices[0].message.content}')
    
    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            print(f'호출된 함수: {tool_call.function.name}')
            print(f'함수 인자: {tool_call.function.arguments}')
            
            # 동적으로 함수 호출
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                result = function_to_call(**function_args)
                print(f'실행 결과: {result}')
                print("-" * 40)
            else:
                print(f"함수 {function_name}을 찾을 수 없습니다.")

# 다양한 요청 테스트 예시
test_messages = [
    "서울 날씨 어때?",
    "지금 몇 시야?", 
    "10 더하기 25는?",
    "파이썬에 대해 알려줘"
]

for msg in test_messages:
    print(f"\n질문: {msg}")
    print("=" * 50)
    process_user_request(msg) 


질문: 서울 날씨 어때?
응답 데이터: None
호출된 함수: get_weather
함수 인자: {"location":"서울"}
실행 결과: 맑고 기온은 24도입니다.
----------------------------------------

질문: 지금 몇 시야?
응답 데이터: None
호출된 함수: get_time
함수 인자: {"timezone":"Asia/Seoul"}
실행 결과: Asia/Seoul 기준 현재 시간: 2025-06-13 09:35:01
----------------------------------------

질문: 10 더하기 25는?
응답 데이터: None
호출된 함수: calculate
함수 인자: {"operation":"add","a":10,"b":25}
실행 결과: 10 + 25 = 35
----------------------------------------

질문: 파이썬에 대해 알려줘
응답 데이터: 파이썬(Python)은 쉽고 강력한 프로그래밍 언어입니다. 1991년 귀도 반 로섬(Guido van Rossum)이 개발했으며, 현재는 전 세계적으로 널리 사용되고 있습니다. 파이썬의 주요 특징은 다음과 같습니다:

### 1. 문법이 간단하고 읽기 쉬움
파이썬은 코드가 영어 문장처럼 읽히기 때문에 초보자도 쉽게 배울 수 있습니다.

### 2. 다양한 용도
웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화, 게임 개발 등 다양한 분야에서 사용됩니다.

### 3. 방대한 라이브러리
수많은 오픈소스 라이브러리와 프레임워크(예: Django, Flask, Pandas, NumPy, TensorFlow 등)가 있어 개발이 편리합니다.

### 4. 인터프리터 언어
코드를 한 줄씩 실행할 수 있어 디버깅과 테스트가 쉽습니다.

### 5. 크로스 플랫폼
윈도우, 맥, 리눅스 등 다양한 운영체제에서 사용할 수 있습니다.

---

#### 예시 코드

```python
print("Hello, World!")
```

이 코

In [24]:
import json
from datetime import datetime


##########################################################################################
########################## step 1 tool로 정의할 function 정의하기 #########################
##########################################################################################

def get_time(timezone="Asia/Seoul"):
    # 간단한 시간 조회 함수
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"{timezone} 기준 현재 시간: {current_time}"
  
##########################################################################################
########################## step 2 tool 정의하기 ##########################################
##########################################################################################

# 여러 개의 도구 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "현재 시간을 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "시간대 (기본값: Asia/Seoul)",
                        "default": "Asia/Seoul"
                    }
                },
                "required": []
            }
        }
    }
]

##########################################################################################
########################## step 3 tool 불러오기 ##########################################
##########################################################################################

user_message = input("메시지를 입력하세요: ")
messages = [{"role": "user", "content": user_message}]
response = client.chat.completions.create(
        model=default_model,
        messages=messages,
        temperature=0.6,
        tools=tools,
        tool_choice="auto",
        stream=True
)

for chunk in response:
    print(chunk.choices[0].delta.content, end="", flush=True) 


# 사용자 입력 초기화
user_message = ""



NoneNoneNoneNoneNoneNoneNoneNoneNoneNone